In [1]:
import re
import requests
from bs4 import BeautifulSoup

In [20]:
import pandas as pd
df = pd.DataFrame({'Автор':[],'Название картины':[], 'Ссылка на картину':[],'Описание':[],'Год':[], 'Размер':[],
                   'Материал':[], 'Техника':[], 'Жанр':[], 'Стиль':[], 'Галерея':[]})

In [3]:
session = requests.Session()

In [21]:
url = 'https://opisanie-kartin.ru/'
page = session.get(url)
soup = BeautifulSoup(page.text, "html.parser")

In [23]:
artists_menu_items = soup.find_all('ul', class_="sub-menu")
artists_menu_items[:2]

[<ul class="sub-menu">
 <li class="menu-item menu-item-type-taxonomy menu-item-object-category first-menu-item menu-item-48210" id="menu-item-48210"><a href="https://opisanie-kartin.ru/category/ajvazovskij-ivan/">Айвазовский Иван</a></li>
 <li class="menu-item menu-item-type-taxonomy menu-item-object-category menu-item-48211" id="menu-item-48211"><a href="https://opisanie-kartin.ru/category/alekseev-fedor/">Алексеев Федор</a></li>
 <li class="menu-item menu-item-type-taxonomy menu-item-object-category menu-item-48212" id="menu-item-48212"><a href="https://opisanie-kartin.ru/category/altdorfer-albreht/">Альтдорфер Альбрехт</a></li>
 <li class="menu-item menu-item-type-taxonomy menu-item-object-category menu-item-48213" id="menu-item-48213"><a href="https://opisanie-kartin.ru/category/argunov-ivan/">Аргунов Иван</a></li>
 <li class="menu-item menu-item-type-taxonomy menu-item-object-category menu-item-48214" id="menu-item-48214"><a href="https://opisanie-kartin.ru/category/arhipov-abram/

In [24]:
dict_names = {}

In [ ]:
for artist_item in artists_menu_items:
    letter_artists = artist_item.find_all('a')
    for link in letter_artists:
        artist = link['href']
        author_en = re.findall(r'(?<=https://opisanie-kartin.ru/category/).*?(?=/)', artist)
        author_en = re.sub(r'\"', '', str(author_en))
        author_en = re.sub(r'\[', '', str(author_en))
        author_en = re.sub(r'\]', '', str(author_en))
        author_en = author_en.strip('\'')
        print(author_en)
        author_ru = str(link.text)
        dict_names[author_en] = author_ru

In [26]:
dict_names['ajvazovskij-ivan']

'Айвазовский Иван'

In [27]:
Artist_Links = []

In [28]:
for Link in soup.body.findAll('a'):
  if re.match('https://opisanie-kartin.ru/category/', Link['href']) != None:
    Artist_Links.append(Link['href'])

In [29]:
 Artist_Links[:10]

['https://opisanie-kartin.ru/category/ajvazovskij-ivan/',
 'https://opisanie-kartin.ru/category/alekseev-fedor/',
 'https://opisanie-kartin.ru/category/altdorfer-albreht/',
 'https://opisanie-kartin.ru/category/argunov-ivan/',
 'https://opisanie-kartin.ru/category/arhipov-abram/',
 'https://opisanie-kartin.ru/category/archimboldo-dzhuzeppe/',
 'https://opisanie-kartin.ru/category/bakst-leon/',
 'https://opisanie-kartin.ru/category/baskiya-zhan-mishel/',
 'https://opisanie-kartin.ru/category/benua-aleksandr/',
 'https://opisanie-kartin.ru/category/bilibin-ivan/']

In [30]:
for Link in Artist_Links:
  url = Link
  page = session.get(url)
  soup = BeautifulSoup(page.text, "html.parser")
  Art_Links = []
  for Link in soup.body.findAll('a'):
    if re.match(re.sub('/category', '', url), Link['href']) != None:
      Art_Links.append(Link['href'])

  for Link in set(Art_Links):
      try:
        url = Link
        page = session.get(url)
        soup = BeautifulSoup(page.text, "html.parser")

        INIT = soup.findAll('div', class_='content-headline')[0].text
        art = re.findall(r'(?<=«).*?(?=»)', INIT)[0]

        author_toupd = re.findall(r'(?<=https://opisanie-kartin.ru/).*?(?=/)', Link)[0]
        author = dict_names[author_toupd]
        
        TEXT = soup.findAll('div', class_='entry-content')[0].text
        try:
            description = re.findall(r'[^A-Za-z]*(?=Год написания картины)', TEXT, re.DOTALL)[0]
        except:
            description = ''
        try:
            year = re.findall(r'(?<=Год написания картины:).*?(?=\n)', TEXT, re.DOTALL)[0]
        except:
            year = ''
        try:
            try:
                size = re.findall(r'(?<=Размеры картины:).*?(?=\n)', TEXT, re.DOTALL)[0]
            except:
                size = re.findall(r'(?<=Размеры:).*?(?=\n)', TEXT, re.DOTALL)[0]
        except:
            size = ''
        try:
            material = re.findall(r'(?<=Материал:).*?(?=\n)', TEXT, re.DOTALL)[0]
        except:
            material = ''
        try:
            technique = re.findall(r'(?<=Техника написания:).*?(?=\n)', TEXT, re.DOTALL)[0]
        except:
            technique = ''
        try:
            genre = re.findall(r'(?<=Жанр:).*?(?=\n)', TEXT, re.DOTALL)[0]
        except:
            genre = ''
        try:
            style = re.findall(r'(?<=Стиль:).*?(?=\n)', TEXT, re.DOTALL)[0]
        except:
            style = ''
        try:
            galery = re.findall(r'(?<=Галерея:).*?(?=\n)', TEXT, re.DOTALL)[0]
        except:
            galery = ''
        df = df.append({'Автор':author, 'Название картины':art, 'Ссылка на картину':Link, 'Описание':description,
                        'Год':year, 'Размер':size, 'Материал':material,
                        'Техника':technique, 'Жанр':genre, 'Стиль':style, 
                        'Галерея':galery}, ignore_index=True)
      except:
          pass

In [31]:
def Preprocessing(dfseries, Extended=True):
    dfseries = dfseries.apply(lambda x: re.sub('\xa0', ' ', x))
    dfseries = dfseries.apply(lambda x: x.split(','))
    if Extended == True:
      dfseries = dfseries.apply(lambda x: [y.strip(' .') for y in x])

In [32]:
for Column in ['Стиль', 'Жанр', 'Техника', 'Материал']:
  Preprocessing(df[Column])
for Column in ['Галерея', 'Размер', 'Год']:
  Preprocessing(df[Column], Extended=False)
df['Описание'] = df['Описание'].apply(lambda x: re.sub('\n', ' ', x))

In [33]:
df.to_csv('pictures.csv')

In [34]:
df = pd.read_csv('pictures.csv')

In [35]:
df

,Unnamed: 0,Автор,Название картины,Ссылка на картину,Описание,Год,Размер,Материал,Техника,Жанр,Стиль,Галерея
0,0,Айвазовский Иван,Спокойное море,https://opisanie-kartin.ru/ajvazovskij-ivan/sp...,Время создания картины «Спокойное море» прих...,1863.,45 x 58 см.,холст.,масло.,марина.,романтизм.,нет данных.
1,1,Айвазовский Иван,Неаполитанский залив утром,https://opisanie-kartin.ru/ajvazovskij-ivan/ne...,.,1843.,67 x 100 см.,холст.,масло.,марина.,романтизм.,нет данных.
2,2,Айвазовский Иван,Прощание А.С. Пушкина с морем,https://opisanie-kartin.ru/ajvazovskij-ivan/pr...,«Дивное море написал Айвазовский… И я удосто...,1877.,нет данных.,холст.,масло.,портрет.,романтизм.,"Всероссийский музей А. С. Пушкина, Санкт-Пете..."
3,3,Айвазовский Иван,Смотр Черноморского флота в 1849 году,https://opisanie-kartin.ru/ajvazovskij-ivan/sm...,,1886.,131 x 249 см.,холст.,масло.,марина.,романтизм.,"Центральный военно-морской музей, Санкт-Петер..."
4,4,Айвазовский Иван,Русско-турецкий морской бой при Синопе 18 нояб...,https://opisanie-kartin.ru/ajvazovskij-ivan/ru...,,1853.,331 x 220 см.,холст.,масло.,батализм.,романтизм.,"Центральный военно-морской музей, Санкт-Петер..."
...,...,...,...,...,...,...,...,...,...,...,...,...
914,914,Яблонская Татьяна,Хлеб,https://opisanie-kartin.ru/yablonskaya-tatyana...,"Картина «Хлеб», написанная известной советск...",1949.,нет данных.,холст.,масло.,жанровая живопись.,социалистический реализм.,нет данных.
915,915,Яблонская Татьяна,Утро,https://opisanie-kartin.ru/yablonskaya-tatyana...,Замечательная советская художница Татьяна Яб...,1954.,169 x 110 см.,холст.,масло.,жанровая живопись.,социалистический реализм.,"Государственная Третьяковская галерея, Москва..."
916,916,Яблонская Татьяна,На старте,https://opisanie-kartin.ru/yablonskaya-tatyana...,,1947.,нет данных.,холст.,масло.,жанровая живопись.,социалистический реализм.,нет данных.
917,917,Яблонская Татьяна,Вечер. Старая Флоренция,https://opisanie-kartin.ru/yablonskaya-tatyana...,,1973.,нет данных.,холст.,масло.,жанровая живопись.,"социалистический реализм, импрессионизм.",нет данных.
